## Utility Functions

All the funcitons required in pre processing the image before feeding it to model, is defined here

* pre_processed() - before finding contours we need to process the image 
        1.a first we will convert it into gray image
        1.b then we will introduce blur into image 
        1.c lastly, we will apply thresholding to filter out the image and distinguish various features

* sharpen() - to sharpen the image
* biggest_contour() - Out of all Contours we have found we want to keep the largest one

* reorder() - Consider the box below, the problem with finding contour it is that we are not sure in what order we r getting them, i.e. if the first contour is A for some other image B could be first contour and Since we want to Crop Out the Sudoku from our Initial image we have to have the corners in ordered form. So I created an reorder function that will give me ordered points of our biggest contour
    
                                    A-----------B
                                    |           |
                                    |           |
                                    |           |
                                    C-----------D

* split_boxes() - after warping out our main sudoku now we want to further divide it into smaller boxes to feed that to our ml model. Since img is an array we can simply divide the array in 9 rows and 9 columns and store each box in out return array boxes

* get_pred() - Now we run our CNN model to classify the images 

In [1]:
def pre_processed(img):
    gray_img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) # GRAY IMAGE
    blur_img = cv2.GaussianBlur(gray_img, (3,3) ,1) # Gaussian blur(src,kernel_size(#always odd),border)
    threshold_img = cv2.adaptiveThreshold(gray_img, 255, 1, 1, 9, 1) 
    return threshold_img

In [2]:
def sharpen(img):
    S_Filter = np.array([[-1,-1,-1],
                     [-1,9,-1],
                     [-1,-1,-1]])
    img = cv2.filter2D(img,-1,S_Filter)
    return img

In [3]:
def biggest_contour(contours):
    biggest = np.array([])
    max_area = 0
    for i in contours:
        area = cv2.contourArea(i)
        if area >1000: # removing noise i.e. small areas
            peri = cv2.arcLength(i,True) # perimeter
            approx = cv2.approxPolyDP(i,0.02*peri, True) # find how many corners does it have
            if area>max_area and len(approx) == 4:
                biggest = approx
                max_area = area
    return biggest, max_area

In [4]:
def reorder(contour_pts):
    contour_pts = contour_pts.reshape((4,2))
    ordered_pts = np.zeros((4,1,2),dtype = np.int32)
    add = contour_pts.sum(1)
    ordered_pts[0] = contour_pts[np.argmin(add)]
    ordered_pts[2] = contour_pts[np.argmax(add)]
    diff = np.diff(contour_pts, axis=1)
    ordered_pts[1] = contour_pts[np.argmin(diff)]
    ordered_pts[3] = contour_pts[np.argmax(diff)]
    return ordered_pts

In [5]:
def split_boxes(warped_img):
    row = np.vsplit(warped_img,9)
    boxes = []
    for r in row:
        cols = np.hsplit(r,9)
        for box in cols:
            box = sharpen(box)
            boxes.append(box)
    return boxes

In [6]:
def get_pred(boxes,model):
    size = 32
    numbers = []
    i = 1
    for box in boxes:
#         box = cv2.resize(box,(size,size))
        box = cv2.cvtColor(box, cv2.COLOR_BGR2GRAY)
#         box = cv2.equalizeHist(box)
        box = np.asarray(box)
        box = box.reshape(1,size,size,1)
        box = box/255.0
        pred_prob = model.predict(box)
        print(i,np.argmax(pred_prob), np.max(pred_prob))
        if np.max(pred_prob) <= 0.7:
            pred_num = 0
        else:
            pred_num = np.argmax(pred_prob)
        numbers.append(pred_num)
        i+=1
    return numbers